# 1D CNN on ECG data with transfer learning

This notebook demonstrates a 1D CNN that is trained on ECG data from the MIT-BIH dataset. This project will utilize transfer learning to learn the general patterns of an ECG. Then this final layers of this pre-trained model are re-trained on one specific patient using the first ten minutes of the ECG trace. Then the model will be evaluated using the final 20 minutes of the ECG trace.

Of course the to-be evaluated patient is not included initial learning step, this will prevent any data leakage. The process will be repeated in a leave-one-out-cross-validation fashion.

In [1]:
import math

import datetime
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models

## Final filtering and loading
The dataset needs to have it's final preprocessing steps applied. 
We factorize the labels (convert strings to integers) so it can be used by the model.
Also we exclude any hearthbeats that have a windown length of 160, these are hearthbeats that occur in the beginning and and of a recording

In [2]:
data = pd.read_pickle("splitted_samples.data")

data = data[data['waves'].str.len() == 160]
# factorize labels
factorized_labels = data["beat_type"].factorize()
data["label"] = factorized_labels[0]

In [3]:
summ = data["beat_type"].value_counts()
print("Percentage of normal hearthbeats")
1- (sum(summ) - summ["N"]) / sum(summ)

Percentage of normal hearthbeats


0.6856484950382865

## Data splitting and normalization
Data needs to be splitted and transformed to the correct shape. Then the values are normalized between 0 and 1. It is important to note that that test and train data is seperately normalized, also the two hearth tracks are also seperately normalized

In [4]:
def train_test_split():
    patient_count = np.unique(data["patient"]).shape[0]
    
    for i in range(patient_count):
        train = data[data["patient"] != i]
        test = data[data["patient"] == i]

        train_data, train_labels = np.array(train["waves"].tolist()), np.expand_dims(train["label"].values, axis=1)
        test_data, test_labels = np.array(test["waves"].tolist()), np.expand_dims(test["label"].values, axis=1)
        
        # Normalization between [0,1]
        train_data[:,:,0] = (train_data[:,:,0] - train_data[:,:,0].min()) / (train_data[:,:,0].max() - train_data[:,:,0].min())
        train_data[:,:,1] = (train_data[:,:,1] - train_data[:,:,1].min()) / (train_data[:,:,1].max() - train_data[:,:,1].min())  
        
        test_data[:,:,0] = (test_data[:,:,0] - test_data[:,:,0].min()) / (test_data[:,:,0].max() - test_data[:,:,0].min())  
        test_data[:,:,1] = (test_data[:,:,1] - test_data[:,:,1].min()) / (test_data[:,:,1].max() - test_data[:,:,1].min())  

        yield (train_data, train_labels), (test_data, test_labels)

In [5]:
model = models.Sequential()
model.add(layers.Conv1D(16, 3, input_shape=(160, 2), activation="relu"))
model.add(layers.MaxPool1D(2))
model.add(layers.Conv1D(32, 3, activation="relu"))
model.add(layers.MaxPool1D(2))
model.add(layers.Conv1D(64, 3, activation="relu"))
model.add(layers.MaxPool1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(14, activation="relu"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 158, 16)           112       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 79, 16)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 77, 32)            1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 38, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 64)            6208      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 18, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0

## Evaluation all models with cross validation

In [6]:
patient = 0
for (train_data, train_labels), (test_data, test_labels) in train_test_split():

    model.compile(optimizer="adam",
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=["accuracy"])

    log_dir = "logs/fit/patient" + str(patient) + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


    history = model.fit(train_data, train_labels, epochs=10,
                        validation_data=(test_data, test_labels),
                        callbacks=[tensorboard_callback])
    patient += 1

Epoch 1/10
3327/3327 [==============================] - 19s 6ms/step - loss: 0.7172 - accuracy: 0.8208 - val_loss: 1.3106 - val_accuracy: 0.8719
Epoch 2/10
3327/3327 [==============================] - 19s 6ms/step - loss: 0.2661 - accuracy: 0.9404 - val_loss: 0.8490 - val_accuracy: 0.7574
Epoch 3/10
3327/3327 [==============================] - 19s 6ms/step - loss: 0.2004 - accuracy: 0.9536 - val_loss: 1.7070 - val_accuracy: 0.5075
Epoch 4/10
3327/3327 [==============================] - 19s 6ms/step - loss: 0.1766 - accuracy: 0.9585 - val_loss: 1.9827 - val_accuracy: 0.4193
Epoch 5/10
3327/3327 [==============================] - 20s 6ms/step - loss: 0.1621 - accuracy: 0.9615 - val_loss: 1.7207 - val_accuracy: 0.4739
Epoch 6/10
3327/3327 [==============================] - 19s 6ms/step - loss: 0.1523 - accuracy: 0.9639 - val_loss: 1.6706 - val_accuracy: 0.5834
Epoch 7/10
3327/3327 [==============================] - 19s 6ms/step - loss: 0.1432 - accuracy: 0.9662 - val_loss: 1.6819 - val_ac

3356/3356 [==============================] - 18s 5ms/step - loss: 0.0725 - accuracy: 0.9800 - val_loss: 0.2476 - val_accuracy: 0.9732
Epoch 8/10
3356/3356 [==============================] - 18s 5ms/step - loss: 0.0714 - accuracy: 0.9802 - val_loss: 0.2609 - val_accuracy: 0.9688
Epoch 9/10
3356/3356 [==============================] - 18s 5ms/step - loss: 0.0715 - accuracy: 0.9801 - val_loss: 0.5331 - val_accuracy: 0.8997
Epoch 10/10
3356/3356 [==============================] - 18s 6ms/step - loss: 0.0715 - accuracy: 0.9801 - val_loss: 0.3426 - val_accuracy: 0.9562
Epoch 1/10
3339/3339 [==============================] - 18s 5ms/step - loss: 0.0709 - accuracy: 0.9804 - val_loss: 0.2809 - val_accuracy: 0.9223
Epoch 2/10
3339/3339 [==============================] - 18s 5ms/step - loss: 0.0692 - accuracy: 0.9809 - val_loss: 0.5182 - val_accuracy: 0.8623
Epoch 3/10
3339/3339 [==============================] - 19s 6ms/step - loss: 0.0690 - accuracy: 0.9810 - val_loss: 0.2314 - val_accuracy: 0.

3315/3315 [==============================] - 18s 5ms/step - loss: 0.0579 - accuracy: 0.9830 - val_loss: 0.5589 - val_accuracy: 0.9212
Epoch 4/10
3315/3315 [==============================] - 18s 5ms/step - loss: 0.0570 - accuracy: 0.9830 - val_loss: 1.2453 - val_accuracy: 0.8210
Epoch 5/10
3315/3315 [==============================] - 18s 5ms/step - loss: 0.0573 - accuracy: 0.9828 - val_loss: 0.7706 - val_accuracy: 0.8802
Epoch 6/10
3315/3315 [==============================] - 18s 5ms/step - loss: 0.0574 - accuracy: 0.9826 - val_loss: 0.7769 - val_accuracy: 0.9126
Epoch 7/10
3315/3315 [==============================] - 18s 5ms/step - loss: 0.0569 - accuracy: 0.9830 - val_loss: 0.4367 - val_accuracy: 0.9387
Epoch 8/10
3315/3315 [==============================] - 18s 5ms/step - loss: 0.0574 - accuracy: 0.9829 - val_loss: 0.7268 - val_accuracy: 0.9161
Epoch 9/10
3315/3315 [==============================] - 18s 5ms/step - loss: 0.0567 - accuracy: 0.9831 - val_loss: 0.4444 - val_accuracy: 0.9

3362/3362 [==============================] - 18s 5ms/step - loss: 0.0524 - accuracy: 0.9839 - val_loss: 0.3426 - val_accuracy: 0.9553
Epoch 10/10
3362/3362 [==============================] - 18s 5ms/step - loss: 0.0517 - accuracy: 0.9840 - val_loss: 0.4040 - val_accuracy: 0.9484
Epoch 1/10
3345/3345 [==============================] - 18s 5ms/step - loss: 0.0541 - accuracy: 0.9833 - val_loss: 0.5026 - val_accuracy: 0.9695
Epoch 2/10
3345/3345 [==============================] - 18s 5ms/step - loss: 0.0523 - accuracy: 0.9839 - val_loss: 0.1308 - val_accuracy: 0.9934
Epoch 3/10
3345/3345 [==============================] - 18s 5ms/step - loss: 0.0532 - accuracy: 0.9835 - val_loss: 0.1251 - val_accuracy: 0.9810
Epoch 4/10
3345/3345 [==============================] - 18s 5ms/step - loss: 0.0523 - accuracy: 0.9839 - val_loss: 0.1583 - val_accuracy: 0.9856
Epoch 5/10
3345/3345 [==============================] - 18s 5ms/step - loss: 0.0525 - accuracy: 0.9838 - val_loss: 0.2037 - val_accuracy: 0.

3370/3370 [==============================] - 18s 5ms/step - loss: 0.0481 - accuracy: 0.9848 - val_loss: 0.3729 - val_accuracy: 0.9419
Epoch 6/10
3370/3370 [==============================] - 18s 5ms/step - loss: 0.0490 - accuracy: 0.9848 - val_loss: 0.1923 - val_accuracy: 0.9512
Epoch 7/10
3370/3370 [==============================] - 18s 5ms/step - loss: 0.0476 - accuracy: 0.9851 - val_loss: 0.4012 - val_accuracy: 0.9382
Epoch 8/10
3370/3370 [==============================] - 18s 5ms/step - loss: 0.0483 - accuracy: 0.9847 - val_loss: 0.6976 - val_accuracy: 0.9327
Epoch 9/10
3370/3370 [==============================] - 18s 5ms/step - loss: 0.0472 - accuracy: 0.9850 - val_loss: 0.5298 - val_accuracy: 0.9265
Epoch 10/10
3370/3370 [==============================] - 18s 5ms/step - loss: 0.0484 - accuracy: 0.9848 - val_loss: 0.6800 - val_accuracy: 0.9364
Epoch 1/10
3350/3350 [==============================] - 18s 5ms/step - loss: 0.0494 - accuracy: 0.9843 - val_loss: 0.0148 - val_accuracy: 0.

3365/3365 [==============================] - 18s 5ms/step - loss: 0.0473 - accuracy: 0.9849 - val_loss: 0.8486 - val_accuracy: 0.8542
Epoch 2/10
3365/3365 [==============================] - 18s 5ms/step - loss: 0.0474 - accuracy: 0.9847 - val_loss: 1.4947 - val_accuracy: 0.7572
Epoch 3/10
3365/3365 [==============================] - 18s 5ms/step - loss: 0.0467 - accuracy: 0.9849 - val_loss: 2.0356 - val_accuracy: 0.7045
Epoch 4/10
3365/3365 [==============================] - 18s 5ms/step - loss: 0.0479 - accuracy: 0.9846 - val_loss: 1.1048 - val_accuracy: 0.8117
Epoch 5/10
3365/3365 [==============================] - 18s 5ms/step - loss: 0.0481 - accuracy: 0.9848 - val_loss: 0.9658 - val_accuracy: 0.8514
Epoch 6/10
3365/3365 [==============================] - 18s 5ms/step - loss: 0.0474 - accuracy: 0.9848 - val_loss: 0.7438 - val_accuracy: 0.8803
Epoch 7/10
3365/3365 [==============================] - 18s 5ms/step - loss: 0.0472 - accuracy: 0.9849 - val_loss: 2.0368 - val_accuracy: 0.6

3334/3334 [==============================] - 18s 5ms/step - loss: 0.0445 - accuracy: 0.9855 - val_loss: 0.8364 - val_accuracy: 0.9786
Epoch 8/10
3334/3334 [==============================] - 18s 5ms/step - loss: 0.0456 - accuracy: 0.9853 - val_loss: 0.2726 - val_accuracy: 0.9807
Epoch 9/10
3334/3334 [==============================] - 18s 5ms/step - loss: 0.0453 - accuracy: 0.9852 - val_loss: 0.5289 - val_accuracy: 0.9691
Epoch 10/10
3334/3334 [==============================] - 18s 5ms/step - loss: 0.0443 - accuracy: 0.9855 - val_loss: 0.5089 - val_accuracy: 0.9815
Epoch 1/10
3373/3373 [==============================] - 18s 5ms/step - loss: 0.0457 - accuracy: 0.9851 - val_loss: 0.0131 - val_accuracy: 0.9980
Epoch 2/10
3373/3373 [==============================] - 18s 5ms/step - loss: 0.0449 - accuracy: 0.9853 - val_loss: 0.0034 - val_accuracy: 0.9987
Epoch 3/10
3373/3373 [==============================] - 18s 5ms/step - loss: 0.0465 - accuracy: 0.9850 - val_loss: 0.0098 - val_accuracy: 0.

3354/3354 [==============================] - 18s 5ms/step - loss: 0.0470 - accuracy: 0.9849 - val_loss: 1.0017 - val_accuracy: 0.9143
Epoch 4/10
3354/3354 [==============================] - 18s 5ms/step - loss: 0.0441 - accuracy: 0.9854 - val_loss: 2.4603 - val_accuracy: 0.8131
Epoch 5/10
3354/3354 [==============================] - 18s 5ms/step - loss: 0.0464 - accuracy: 0.9850 - val_loss: 1.8762 - val_accuracy: 0.8385
Epoch 6/10
3354/3354 [==============================] - 18s 5ms/step - loss: 0.0457 - accuracy: 0.9850 - val_loss: 2.4612 - val_accuracy: 0.8131
Epoch 7/10
3354/3354 [==============================] - 18s 5ms/step - loss: 0.0449 - accuracy: 0.9854 - val_loss: 1.2582 - val_accuracy: 0.8847
Epoch 8/10
3354/3354 [==============================] - 18s 5ms/step - loss: 0.0446 - accuracy: 0.9853 - val_loss: 1.9113 - val_accuracy: 0.8112
Epoch 9/10
3354/3354 [==============================] - 18s 5ms/step - loss: 0.0453 - accuracy: 0.9851 - val_loss: 1.5098 - val_accuracy: 0.8

3341/3341 [==============================] - 17s 5ms/step - loss: 0.0435 - accuracy: 0.9855 - val_loss: 0.4115 - val_accuracy: 0.9739
Epoch 10/10
3341/3341 [==============================] - 18s 5ms/step - loss: 0.0459 - accuracy: 0.9850 - val_loss: 0.5127 - val_accuracy: 0.9672
Epoch 1/10
3338/3338 [==============================] - 18s 5ms/step - loss: 0.0437 - accuracy: 0.9854 - val_loss: 9.0357 - val_accuracy: 0.8943
Epoch 2/10
3338/3338 [==============================] - 18s 5ms/step - loss: 0.0452 - accuracy: 0.9853 - val_loss: 6.7227 - val_accuracy: 0.9045
Epoch 3/10
3338/3338 [==============================] - 18s 5ms/step - loss: 0.0453 - accuracy: 0.9853 - val_loss: 9.9087 - val_accuracy: 0.8720
Epoch 4/10
3338/3338 [==============================] - 18s 5ms/step - loss: 0.0433 - accuracy: 0.9856 - val_loss: 10.1207 - val_accuracy: 0.8301
Epoch 5/10
3338/3338 [==============================] - 18s 5ms/step - loss: 0.0446 - accuracy: 0.9855 - val_loss: 5.8969 - val_accuracy: 0